# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [25]:
import logging
import csv
import os 
import shutil

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np 
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core import Experiment, Model, Environment
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace, Dataset, Datastore 
from azureml.pipeline.steps import AutoMLStep
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn



print("SDK version: ", azureml.core.VERSION)

SDK version:  1.33.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [26]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'heart-failure-pred'

experiment=Experiment(ws, experiment_name)
print(experiment)

found = False
ds = 'workspaceblobstore'
key = 'heart-failure-pred'
description_text = "Heart failure prediction dataset for capstone project"

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key]
if not found:
    data = "https://mlstrg158340.blob.core.windows.net/azureml-blobstore-e1a969f3-969a-4bbd-a15c-43e00c8da638/UI/09-15-2021_091742_UTC/heart_failure_clinical_records_dataset.csv"
    dataset = Dataset.Tabular.from_delimited_files(data)
    dataset = dataset.register(workspace=ws,
                               name=key,
                               description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

Experiment(Name: heart-failure-pred,
Workspace: quick-starts-ws-158550)


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00
mean,60.83,0.43,581.84,0.42,38.08,0.35,263358.03,1.39,136.63,0.65,0.32,130.26,0.32
std,11.89,0.50,970.29,0.49,11.83,0.48,97804.24,1.03,4.41,0.48,0.47,77.61,0.47
min,40.00,0.00,23.00,0.00,14.00,0.00,25100.00,0.50,113.00,0.00,0.00,4.00,0.00
25%,51.00,0.00,116.50,0.00,30.00,0.00,212500.00,0.90,134.00,0.00,0.00,73.00,0.00
50%,60.00,0.00,250.00,0.00,38.00,0.00,262000.00,1.10,137.00,1.00,0.00,115.00,0.00
75%,70.00,1.00,582.00,1.00,45.00,1.00,303500.00,1.40,140.00,1.00,1.00,203.00,1.00
max,95.00,1.00,7861.00,1.00,80.00,1.00,850000.00,9.40,148.00,1.00,1.00,285.00,1.00


In [27]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.00,0,582,0,20,1,265000.00,1.90,130,1,0,4,1
1,55.00,0,7861,0,38,0,263358.03,1.10,136,1,0,6,1
2,65.00,0,146,0,20,0,162000.00,1.30,129,1,1,7,1
3,50.00,1,111,0,20,0,210000.00,1.90,137,1,0,7,1
4,65.00,1,160,1,20,0,327000.00,2.70,116,0,0,8,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [29]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 30,
                   "max_concurrent_iterations": 5,
                   "primary_metric": "accuracy"}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task="classification",
                              compute_target="project-cluster",
                              training_data=dataset,
                              label_column_name="DEATH_EVENT",
                              n_cross_validations=5,
                              **automl_settings)

In [30]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-pred,AutoML_3f729b1f-f534-43a2-b3d0-a44a77c5bbde,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [31]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-pred,AutoML_3f729b1f-f534-43a2-b3d0-a44a77c5bbde,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

{'runId': 'AutoML_3f729b1f-f534-43a2-b3d0-a44a77c5bbde',
 'target': 'project-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-09-17T11:51:02.72401Z',
 'endTimeUtc': '2021-09-17T12:07:27.557484Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'project-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-failure-pred","subscription_id":"f9d5a085-54dc-4215-9ba6-dad5d86e60a0","resource_group":"aml-quickstarts-158550","workspace_name":"quick-starts-ws-158550","region":"southcentralus","compute_target":"project-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metri

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [32]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)
print(remote_run.get_metrics())

Run(Experiment: heart-failure-pred,
Id: AutoML_3f729b1f-f534-43a2-b3d0-a44a77c5bbde_58,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
)), ('svcwrapper', SVCWrapper(C=16.768329368110066, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf', max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001, verbose=False))], verbose=False)), ('43', Pipeline(memory=None, steps=[('robustscaler', RobustScaler(copy=True, quantile_range=[25, 75], with_centering=True, with_scaling=False)), ('lightgbmclassifier', Ligh

In [33]:
#TODO: Save the best model
model = remote_run.register_model(model_name='hf-pred-best-autml-model')
print(model.name, model.version, model.id, sep='\n')

hf-pred-best-autml-model
6
hf-pred-best-autml-model:6


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [34]:
from azureml.train.automl.run import AutoMLRun
run_id = "AutoML_3f729b1f-f534-43a2-b3d0-a44a77c5bbde_58"
best_run = AutoMLRun(experiment, run_id)


In [36]:
script_file = 'score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

inference_config = InferenceConfig(entry_script='score.py', environment=best_run.get_environment())
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
aci_service_name = 'heart-failure-pred'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

heart-failure-pred
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-09-17 12:09:53+00:00 Creating Container Registry if not exists.
2021-09-17 12:09:53+00:00 Registering the environment.
2021-09-17 12:09:54+00:00 Use the existing image.
2021-09-17 12:09:54+00:00 Generating deployment configuration.
2021-09-17 12:09:55+00:00 Submitting deployment to compute.
2021-09-17 12:09:58+00:00 Checking the status of deployment heart-failure-pred..
2021-09-17 12:14:37+00:00 Checking the status of inference endpoint heart-failure-pred.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [37]:
aci_service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [38]:
test_data = df.sample(10)
test_label = test_data.pop('DEATH_EVENT')

In [39]:
import json 
import requests 

test_sample = json.dumps({'data': test_data.to_dict(orient='records')})
response = aci_service.run(input_data=test_sample)
response

'{"result": [1, 1, 1, 0, 0, 0, 0, 0, 0, 0]}'

In [40]:
#testing ground truth

test_label

195    1
230    1
61     1
98     0
111    0
221    0
95     0
107    0
211    0
101    0
Name: DEATH_EVENT, dtype: int64

In [42]:
%run endpoint.py

{"result": [1, 1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [43]:
print(aci_service.get_logs())

2021-09-17T12:20:19,402028600+00:00 - nginx/run 
2021-09-17T12:20:19,444497900+00:00 - iot-server/run 
2021-09-17T12:20:19,459491600+00:00 - rsyslog/run 
2021-09-17T12:20:19,443293100+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
rsyslogd: /azureml-envs/azureml_77e992fdaf2b83b0c81ee1c90ded8d17/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-09-17T12:20:19,738457500+00:00 - iot-server/finish 1 0
2021-09-17T12:20:19,740378600+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (57)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 85
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-09-17 12:20:22,878 | root | INFO | Starting up app insights client
logging socket was 

In [44]:
aci_service.delete()